# Welcome to the Image Blending lab!

Let's just warm up, declaring a very simple function.

We can call the function with a name as an argument, or leave it blank to get the default value
```py
hello("student")
hello()
```

Let's try. Go to the section below, and press `Shift + Enter`. It will define the function, and call it. If you want, you can edit the contents of the block and run it again.

The output should be printed below.

In [ ]:
def hello(who="world"):
    print(f"Hello, {who}!")

hello()
hello("student")

Wow, that was easy. Now we can begin with the lab.

## Image Blending

First, we must import libraries

In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = [10, 10]

After we have done this, we can continue with more code, the `import` from the previous block is still valid.

For convenience, we define a function for displaying an image in the notebook before we get to work:

In [ ]:
def showResultOpenCV(title, img):
        cv2.namedWindow(title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
        cv2.imshow(title, img)
        cv2.waitKey(0)

def showResult(title, img):
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.show()

def showResultsSideBySide(title1, img1, title2, img2):
        # Display the original and the transformed image
        axes = plt.subplots(1, 2)[1]
        ax1, ax2 = axes
        ax1.set_title(title1)
        ax2.set_title(title2)
        ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        ax2.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        plt.show()

First we load an image, and the convert it to a floating point datatype. We also divide it by `255`, the maximum pixel value for a channel. That way, we get an image with pixel intensities between 0 and 1.

In [ ]:
img_01_fname = "../../tiger.png"
img_01 = cv2.imread(img_01_fname, cv2.IMREAD_UNCHANGED)
img_01 = np.float32(img_01)*(1.0/255.0)


After you have managed to load the image successfully, let's take a look at that tiger 🐅

In [ ]:
showResult(img_01_fname, img_01)

Whoa! 🐯

Load one more image and take a look at it:

In [ ]:
img_02_fname = "../../white_tiger.png"
img_02 = cv2.imread(img_02_fname, cv2.IMREAD_UNCHANGED)
img_02 = np.float32(img_02)*(1.0/255.0)

showResult(img_02_fname, img_02)

We will now do a "dumb" blending, by just adding the two images together.

Since pixel values must be between 0 and 1, we divide the sum by 2, which is the theoretical maximum sum of two pixels.

In [ ]:
def dumbBlend(img1, img2):
        return (img1 + img2)/2.

dumb_blend = dumbBlend(img_01, img_02)
showResult('dumb_blend', dumb_blend)

Wow, that looks really dumb! 😅

We want the left half of the result to be `tiger`, and the right half to be `white_tiger`, so we need a way to control _where_ and _how much_ each image contribute to the final result.
We can achieve this if we multiply each image with a mask before adding them together.

For the `tiger`, we want max contribution to the left and none to the right, so we create a mask where all pixels on the left half are 1, and the right side are 0.
For the `white_tiger`, we want the opposite and can use the inverted mask.

We call the mask `weights`, since we in practice compute a weighted sum of images.

In [ ]:
def linearBlend(img1, img2, weights):
        return np.multiply(img1, weights) + np.multiply(img2, 1.-weights)

# The weight image is the same size as the input image
weights = np.zeros(img_01.shape, dtype=np.float32)
half_image_width = int(weights.shape[1]/2)

# In the left half of the image, we set all pixels to {1,1,1}
weights[:, :half_image_width] = (1., 1., 1.)

linear_blend = linearBlend(img_01, img_02, weights)
showResultsSideBySide('weights', weights, 'linear_blend', linear_blend)


So far, so good, but to call this a "blend" would be an overstatement. Let's blur the transition between 0 and 1!

Feel free to experiment with the ramp width.

In [ ]:
ramp_width = 150
blurred_weights = cv2.blur(weights, (ramp_width+1, ramp_width+1))

linear_blend = linearBlend(img_01, img_02, blurred_weights)

showResult('linear_blend', linear_blend)

Well, we're getting there, but we can do better.

We are going to use Laplace blending, as you have learned in this week's lecture.

In [ ]:
def gaussPyr(img):
    pyr = [img]
    while pyr[-1].shape[1] > 16:
        pyr.append(cv2.pyrDown(pyr[-1]))
    return pyr

def laplacePyr(img):
    pyr = gaussPyr(img)
    for i in range(len(pyr)-1):
        pyr[i] = pyr[i] - cv2.pyrUp(pyr[i+1],dstsize=pyr[i].shape[0:2])
    return pyr

In [ ]:
pyr_mask = gaussPyr(weights)
pyr_img1 = laplacePyr(img_01)
pyr_img2 = laplacePyr(img_02)
pyr_blend = []
for img1, img2, mask in zip(pyr_img1, pyr_img2, pyr_mask):
    pyr_blend.append(
        linearBlend(img1, img2, mask)
    )

def collapsePyr(pyr):
    for i, img in reversed(list(enumerate(pyr[:-1]))):
        pyr[i] = pyr[i] + cv2.pyrUp(pyr[i+1], dstsize=pyr[i].shape[0:2])
    
    return pyr[0]

blend = collapsePyr(pyr_blend)
showResult('blend', blend)

Now that's just beautiful!